In [91]:
%load_ext micropython_magic
%reload_ext micropython_magic

The micropython_magic extension is already loaded. To reload it, use:
  %reload_ext micropython_magic


In [92]:
# %%micropython
from machine import RTC
from mp_libs import memory
from mp_libs import mptest
from mp_libs.memory import BackupRAM, BackupList, BackupDict

PREPOP_ELEMS = 10
PREPOP_CHUNK_SIZE = 128
NAME_UNSIGNED_CHAR = "uc"
VAL_UNSIGNED_CHAR = 50
NAME_SIGNED_CHAR = "sc"
VAL_SIGNED_CHAR = -50
NAME_UNSIGNED_SHORT = "ush"
VAL_UNSIGNED_SHORT = 0xFF00
NAME_SIGNED_SHORT = "sh"
VAL_SIGNED_SHORT = -0x400
NAME_UNSIGNED_INT = "ui"
VAL_UNSIGNED_INT = 0xFFFF_0000
NAME_SIGNED_INT = "i"
VAL_SIGNED_INT = -0x00FF_0000
NAME_UNSIGNED_LONG = "ul"
VAL_UNSIGNED_LONG = 0xFFFF_0000
NAME_SIGNED_LONG = "l"
VAL_SIGNED_LONG = -0x00FF_0000
NAME_FLOAT = "flt"
VAL_FLOAT = 3.14
NAME_STR = "str"
VAL_STR = "Testing to the maxxxxxxxxxxxxxxxxxxxxxxxxxx"


def setup():
    pass

def teardown():
    pass

def pre_populate_backup_ram(backup_cls):
    for start_idx in range(0, RTC.MEM_SIZE - PREPOP_CHUNK_SIZE, PREPOP_CHUNK_SIZE):
        backup_ram = backup_cls(start_idx, PREPOP_CHUNK_SIZE, reset=True)
        backup_ram.add_element(NAME_UNSIGNED_CHAR, "B", VAL_UNSIGNED_CHAR)
        backup_ram.add_element(NAME_SIGNED_CHAR, "b", VAL_SIGNED_CHAR)
        backup_ram.add_element(NAME_UNSIGNED_SHORT, "H", VAL_UNSIGNED_SHORT)
        backup_ram.add_element(NAME_SIGNED_SHORT, "h", VAL_SIGNED_SHORT)
        backup_ram.add_element(NAME_UNSIGNED_INT, "I", VAL_UNSIGNED_INT)
        backup_ram.add_element(NAME_SIGNED_INT, "i", VAL_SIGNED_INT)
        backup_ram.add_element(NAME_UNSIGNED_LONG, "L", VAL_UNSIGNED_LONG)
        backup_ram.add_element(NAME_SIGNED_LONG, "l", VAL_SIGNED_LONG)
        backup_ram.add_element(NAME_FLOAT, "f", VAL_FLOAT)
        backup_ram.add_element(NAME_STR, f"{len(VAL_STR)}s", VAL_STR)

@mptest.parametrize("backup_cls", [BackupRAM, BackupList, BackupDict])
def test_backup_ram_init_all(backup_cls):
    BackupRAM.reset_rtc()
    backup_ram = backup_cls(0, RTC.MEM_SIZE)
    assert backup_ram._get_magic() == memory.MAGIC_NUM, f"Magic: {backup_ram._get_magic()}"
    assert backup_ram._get_num_elems() == 0, f"Num elems: {backup_ram._get_num_elems()}"
    assert backup_ram._get_free_index() == memory.ELEMENTS_OFFSET, f"Free idx: {backup_ram._get_free_index()}"

@mptest.parametrize("backup_cls", [BackupRAM, BackupList, BackupDict])
def test_backup_ram_init_chunks(backup_cls):
    BackupRAM.reset_rtc()
    for start_idx in range(0, RTC.MEM_SIZE - 100, 100):
        backup_ram = backup_cls(start_idx, 100)
        assert backup_ram._get_magic() == memory.MAGIC_NUM, f"Idx: {start_idx}, Magic: {backup_ram._get_magic()}"
        assert backup_ram._get_num_elems() == 0, f"Idx: {start_idx}, Num elems: {backup_ram._get_num_elems()}"
        assert backup_ram._get_free_index() == start_idx + memory.ELEMENTS_OFFSET, f"Idx: {start_idx}, Free idx: {backup_ram._get_free_index()}"

@mptest.parametrize("backup_cls", [BackupRAM, BackupList, BackupDict])
def test_backup_ram_add_get_element(backup_cls):
    pre_populate_backup_ram(backup_cls)
    for start_idx in range(0, RTC.MEM_SIZE - PREPOP_CHUNK_SIZE, PREPOP_CHUNK_SIZE):
        backup_ram = backup_cls(start_idx, PREPOP_CHUNK_SIZE)
        assert backup_ram._get_num_elems() == PREPOP_ELEMS
        assert backup_ram._get_free_index() == start_idx + PREPOP_CHUNK_SIZE
        assert backup_ram.get_element(NAME_UNSIGNED_CHAR) == VAL_UNSIGNED_CHAR
        assert backup_ram.get_element(NAME_SIGNED_CHAR) == VAL_SIGNED_CHAR
        assert backup_ram.get_element(NAME_UNSIGNED_SHORT) == VAL_UNSIGNED_SHORT
        assert backup_ram.get_element(NAME_SIGNED_SHORT) == VAL_SIGNED_SHORT
        assert backup_ram.get_element(NAME_UNSIGNED_INT) == VAL_UNSIGNED_INT
        assert backup_ram.get_element(NAME_SIGNED_INT) == VAL_SIGNED_INT
        assert backup_ram.get_element(NAME_UNSIGNED_LONG) == VAL_UNSIGNED_LONG
        assert backup_ram.get_element(NAME_SIGNED_LONG) == VAL_SIGNED_LONG
        assert backup_ram.get_element(NAME_FLOAT) == VAL_FLOAT
        assert backup_ram.get_element(NAME_STR) == VAL_STR

@mptest.parametrize("backup_cls", [BackupRAM, BackupList, BackupDict])
def test_backup_ram_set_element(backup_cls):
    pre_populate_backup_ram(backup_cls)
    for start_idx in range(0, RTC.MEM_SIZE - PREPOP_CHUNK_SIZE, PREPOP_CHUNK_SIZE):
        backup_ram = backup_cls(start_idx, PREPOP_CHUNK_SIZE)
        backup_ram.set_element(NAME_UNSIGNED_CHAR, VAL_UNSIGNED_CHAR + 1)
        backup_ram.set_element(NAME_SIGNED_CHAR, VAL_SIGNED_CHAR + 1)
        backup_ram.set_element(NAME_UNSIGNED_SHORT, VAL_UNSIGNED_SHORT + 1)
        backup_ram.set_element(NAME_SIGNED_SHORT, VAL_SIGNED_SHORT + 1)
        backup_ram.set_element(NAME_UNSIGNED_INT, VAL_UNSIGNED_INT + 1)
        backup_ram.set_element(NAME_SIGNED_INT, VAL_SIGNED_INT + 1)
        backup_ram.set_element(NAME_UNSIGNED_LONG, VAL_UNSIGNED_LONG + 1)
        backup_ram.set_element(NAME_SIGNED_LONG, VAL_SIGNED_LONG + 1)
        backup_ram.set_element(NAME_FLOAT, VAL_FLOAT + 1)
        backup_ram.set_element(NAME_STR, "Testing to the mazzzzzzzzzzzzzzzzzzzzzzzzzz")

    for start_idx in range(0, RTC.MEM_SIZE - PREPOP_CHUNK_SIZE, PREPOP_CHUNK_SIZE):
        backup_ram = backup_cls(start_idx, PREPOP_CHUNK_SIZE)
        assert backup_ram._get_num_elems() == PREPOP_ELEMS
        assert backup_ram._get_free_index() == start_idx + PREPOP_CHUNK_SIZE
        assert backup_ram.get_element(NAME_UNSIGNED_CHAR) == VAL_UNSIGNED_CHAR + 1
        assert backup_ram.get_element(NAME_SIGNED_CHAR) == VAL_SIGNED_CHAR + 1
        assert backup_ram.get_element(NAME_UNSIGNED_SHORT) == VAL_UNSIGNED_SHORT + 1
        assert backup_ram.get_element(NAME_SIGNED_SHORT) == VAL_SIGNED_SHORT + 1
        assert backup_ram.get_element(NAME_UNSIGNED_INT) == VAL_UNSIGNED_INT + 1
        assert backup_ram.get_element(NAME_SIGNED_INT) == VAL_SIGNED_INT + 1
        assert backup_ram.get_element(NAME_UNSIGNED_LONG) == VAL_UNSIGNED_LONG + 1
        assert backup_ram.get_element(NAME_SIGNED_LONG) == VAL_SIGNED_LONG + 1
        assert backup_ram.get_element(NAME_FLOAT) == VAL_FLOAT + 1
        assert backup_ram.get_element(NAME_STR) == "Testing to the mazzzzzzzzzzzzzzzzzzzzzzzzzz"

@mptest.parametrize("backup_cls", [BackupRAM, BackupList, BackupDict])
def test_backup_ram_reset(backup_cls):
    pre_populate_backup_ram(backup_cls)
    backup_ram = backup_cls()
    assert backup_ram._get_num_elems() == PREPOP_ELEMS, "1"
    assert backup_ram._get_free_index() == PREPOP_CHUNK_SIZE, f"expected: {PREPOP_CHUNK_SIZE}, actual: {backup_ram._get_free_index()}"
    backup_ram.reset()
    assert backup_ram._get_num_elems() == 0, "3"
    assert backup_ram._get_free_index() == memory.ELEMENTS_OFFSET, "4"

@mptest.parametrize("backup_cls", [BackupRAM, BackupList, BackupDict])
def test_backup_ram_reset_rtc(backup_cls):
    BackupRAM.reset_rtc()
    rtc = RTC()
    for i in range(len(rtc)):
        assert rtc[i] == 0, "5"

    backup_ram = backup_cls()
    assert backup_ram._get_num_elems() == 0, "6"
    assert backup_ram._get_free_index() == memory.ELEMENTS_OFFSET, "7"

@mptest.parametrize("backup_cls", [BackupRAM, BackupList, BackupDict])
def test_backup_ram_size_too_small(backup_cls):
    BackupRAM.reset_rtc()
    with mptest.raises(MemoryError):
        backup_ram = backup_cls(0, memory.ELEMENTS_OFFSET - 2)

@mptest.parametrize("backup_cls", [BackupRAM, BackupList, BackupDict])
def test_backup_ram_overflow_max(backup_cls):
    BackupRAM.reset_rtc()
    backup_ram = backup_cls(RTC.MEM_SIZE - 20)
    with mptest.raises(MemoryError):
        backup_ram.add_element("overflow", "i", 100)

@mptest.parametrize("backup_cls", [BackupRAM, BackupList, BackupDict])
def test_backup_ram_overflow_chunk(backup_cls):
    pre_populate_backup_ram(backup_cls)
    backup_ram = backup_cls(0, PREPOP_CHUNK_SIZE)
    with mptest.raises(MemoryError):
        backup_ram.add_element("uhoh", "B", 100)

@mptest.parametrize("backup_cls", [BackupRAM, BackupList, BackupDict])
def test_backup_ram_invalid_magic_recover(backup_cls):
    pre_populate_backup_ram(backup_cls)
    backup_ram = backup_cls(0)
    backup_ram.add_element("uhohuhoh", "B", 100)
    assert backup_ram._get_num_elems() == PREPOP_ELEMS + 1

    backup_ram_ii = BackupRAM(100)
    assert backup_ram_ii._get_num_elems() == 0


In [93]:
# %%micropython
from machine import RTC
from mp_libs import memory
from mp_libs import mptest
from mp_libs.memory import BackupList

def pre_populate_backup_list():
    backup_list = BackupList(0, RTC.MEM_SIZE, reset=True)
    elem_size = 251 + 4
    num_elems = (RTC.MEM_SIZE - memory.ELEMENTS_OFFSET) // elem_size

    for i in range(num_elems):
        char = chr(ord('a') + i)
        backup_list.append(char * 251)

def test_backup_list_iterator():
    pre_populate_backup_list()
    backup_list = BackupList()

    for i, s in enumerate(backup_list):
        char = chr(ord('a') + i)
        assert s == char * 251

def test_backup_list_getitem():
    pre_populate_backup_list()
    backup_list = BackupList()

    for i, _ in enumerate(backup_list):
        char = chr(ord('a') + i)
        assert backup_list[i] == char * 251

def test_backup_list_setitem():
    pre_populate_backup_list()
    backup_list = BackupList()

    for i in range(len(backup_list)):
        char = chr(ord('A') + i)
        backup_list[i] = char * 251

    for i in range(len(backup_list)):
        char = chr(ord('A') + i)
        assert backup_list[i] == char * 251

def test_backup_list_len():
    pre_populate_backup_list()
    backup_list = BackupList()
    elem_size = 251 + 4
    num_elems = (RTC.MEM_SIZE - memory.ELEMENTS_OFFSET) // elem_size
    assert len(backup_list) == num_elems

    for s in backup_list:
        assert len(s) == 251

def test_backup_list_max_elem_size():
    backup_list = BackupList(0, RTC.MEM_SIZE, reset=True)
    backup_list.append("a" * (memory.ELEMENT_MAX_DATA_LEN + 10))
    assert len(backup_list[0]) == memory.ELEMENT_MAX_DATA_LEN

def test_backup_list_overflow_max():
    pre_populate_backup_list()
    backup_list = BackupList()

    with mptest.raises(MemoryError):
        backup_list.append('s')

    with mptest.raises(MemoryError):
        backup_list.append(500)

def test_backup_list_change_type():
    pre_populate_backup_list()
    backup_list = BackupList()
    with mptest.raises(TypeError):
        backup_list[5] = 5

def test_backup_list_invalid_type():
    backup_list = BackupList(reset=True)
    with mptest.raises(TypeError):
        backup_list.append({'x': 5})

def test_backup_list_clear_if_full():
    pre_populate_backup_list()
    backup_list = BackupList(clear_if_full=True)
    backup_list.append("overflow string")
    assert len(backup_list) == 1
    assert backup_list[0] == "overflow string"

def test_backup_list_clear():
    pre_populate_backup_list()
    backup_list = BackupList()
    backup_list.clear()
    assert len(backup_list) == 0
    assert bool(backup_list) == False

def test_backup_list_copy():
    pre_populate_backup_list()
    backup_list = BackupList()
    copy = backup_list.copy()

    for i, s in enumerate(backup_list):
        assert s == copy[i]


In [94]:
# %%micropython
"""Test Runner"""
from mp_libs.mptest import run

run(globals())


Running 21 tests
.....................
SUCCESS
